In [1]:
import numpy as np # only required to use numpy

In [2]:
class LogisticRegression:
    def __init__(self):
        '''
            textbook matrix shape (row x column)
                x : 2 x 100
                w : 2 x 1
                wTx : 1 x 100
                y : 1 x 100

            this code matrix shape (row x column)
                x : 100 x 2
                w : 2 x 1
                xw : 100 x 1
                y : 100 x 1
        '''
        self.w = np.random.rand(2,1) # weight initialization
        self.b = np.random.rand(1) # bias initialization
        
    def sigmoid(self, z):
        '''
            Activation function of logistic regression
            input :
                z : ? x 1
            output :
                prediction(y_hat) : ? x 1
        '''
        return 1 / (1 + np.exp(-z))

    def predict(self, x, classify=False):
        '''
            Predict using logistic regression
            input : 
                data(x) : ? x 2
                weight(w) : 2 x 1
                bias(b) : 1 x 1 => ? x 1 (broadcast)
            output : 
                prediction(y_hat) : ? x 1
        '''
        if classify == True:
            classifier = np.vectorize(lambda x: 1 if x >= 0.5 else 0)
            return classifier(self.sigmoid(x @ self.w + self.b))
        else:
            return self.sigmoid(x @ self.w + self.b) # numpy broadcast b


    def cost(self, y_hat, y):
        '''
            Cost function of logistic regression
            input : 
                prediction(y_hat) : ? x 1
                label(y) : ? x 1
            output : 
                cost function value : 1 x 1
        '''
        assert len(y_hat) == len(y)

        m = len(y)

        epsilon = 0.000000001 # prevent -inf in log operation

        cost = (-1 / m) * np.sum(y * np.log(y_hat + epsilon) + (1-y) * np.log(1 - y_hat + epsilon))

        return cost

    def gradient_descent(self, x, y, lr):
        y_hat = self.predict(x)
        
        m = len(x)
        
        dz = y_hat - y # dz : ? x 1
        dw = (1 / m) * (x.T @  dz)# x : 2 x ?
        db = (1 / m) * np.sum(dz)
        
        self.w -= lr * dw
        self.b -= lr * db

    def train(self, train_x, train_y, val_x, val_y, lr, iters):
        for i in range(iters):
            prev_w = self.w
            prev_b = self.b
            prev_cost = self.cost(self.predict(val_x, classify=False), val_y)
            
            self.gradient_descent(train_x, train_y, lr) # update w,b
            
            # stop training if cost has increased in validation set : prevent overfitting
            if prev_cost < self.cost(self.predict(val_x, classify=False), val_y): 
                self.w = prev_w
                self.b = prev_b
                print("Iteration:", i ,"Cost:", prev_cost, "Validation Break!")
                break
            
            if i % 10000 == 0:
                print("Iteration:", i ,"Cost:", prev_cost)
                
    def accuracy(self, test_x, test_y):
        y_hat = self.predict(test_x, classify=True)
        diff = y_hat - test_y
        return 1.0 - (np.count_nonzero(diff) / len(diff))

In [ ]:
class OneHiddenLayer:
    pass

In [3]:
# input data

train_x = []
train_y = []
train_file = open("two_moon_0.2/train.txt", "r")
for line in train_file.readlines():
    l = line.rstrip().split()
    train_x.append([float(l[0]), float(l[1])])
    train_y.append([int(l[2])])
train_file.close()
train_x = np.array(train_x)
train_y = np.array(train_y)

val_x = []
val_y = []
val_file = open("two_moon_0.2/val.txt", "r")
for line in val_file.readlines():
    l = line.rstrip().split()
    val_x.append([float(l[0]), float(l[1])])
    val_y.append([int(l[2])])
val_file.close()
val_x = np.array(val_x)
val_y = np.array(val_y)

test_x = []
test_y = []
test_file = open("two_moon_0.2/test.txt", "r")
for line in test_file.readlines():
    l = line.rstrip().split()
    test_x.append([float(l[0]), float(l[1])])
    test_y.append([int(l[2])])
test_file.close()
test_x = np.array(test_x)
test_y = np.array(test_y)

In [4]:
prob_1 = LogisticRegression()
prob_1.train(train_x, train_y, val_x, val_y, lr=0.001, iters=300000)
print("Accuracy:", prob_1.accuracy(test_x, test_y))




Iteration: 0 Cost: 0.7111838841769356
Iteration: 10000 Cost: 0.4150647301456117
Iteration: 20000 Cost: 0.36286690603358407
Iteration: 30000 Cost: 0.3411176904461978
Iteration: 40000 Cost: 0.3294960062480189
Iteration: 50000 Cost: 0.322634845101154
Iteration: 60000 Cost: 0.3184051419421011
Iteration: 70000 Cost: 0.31575790543924215
Iteration: 80000 Cost: 0.314109237651978
Iteration: 90000 Cost: 0.31310996010641107
Iteration: 100000 Cost: 0.31254148756562067
Iteration: 110000 Cost: 0.31226251053185633
Iteration: 120000 Cost: 0.3121794022042423
Iteration: 120633 Cost: 0.31217914424966714 Validation Break!
Accuracy: 0.845
